
---

# 🔒 **9. Security Best Practices**

---

## 🧑‍💻 Least Privilege

```dockerfile
RUN useradd -m app && chown -R app:app /app
USER app
```

```bash
docker run --user 1000:1000 --cap-drop ALL --cap-add NET_BIND_SERVICE myapp
```

* ❌ Avoid `--privileged`
* ✅ Run as **non-root**, drop caps

---

## 🛡️ Lock Down Runtime

```bash
docker run \
  --read-only --tmpfs /tmp \
  --security-opt no-new-privileges:true \
  --pids-limit 200 --memory 512m --cpus 1 \
  myapp
```

* Read-only FS, tmpfs for writes
* Limit **PIDs / RAM / CPU**

---

## 🌐 Network Hygiene

```bash
docker run --network none myjob   # isolate
docker run -p 8080:8080 myapi     # expose only needed ports
```

* Use **user-defined bridge** per stack
* ❌ Never mount Docker socket inside container

---

## 📦 Image Provenance

```dockerfile
FROM python:3.11-slim@sha256:<digest>
RUN apt-get update && apt-get install -y curl \
 && rm -rf /var/lib/apt/lists/*
```

```bash
export DOCKER_CONTENT_TRUST=1   # verify signatures
docker build --pull -t myapp .
```

* ✅ Slim/alpine base
* ✅ Pin by **digest**
* 🔄 Rebuild often

---

## 🔑 Secrets

```bash
docker run --env-file .env myapp     # dev only
# Prod → use K8s/Swarm secrets (not env baked into image)
```

* Add `.env` to **.gitignore**
* Use **mounts** for secret files

---

## 🔍 Scan & Audit

```bash
docker scan myimage     # Snyk
trivy image myimage     # alt popular scanner
```

```bash
docker run --rm --net host --pid host --privileged \
  -v /var/run/docker.sock:/var/run/docker.sock -v /etc:/etc \
  docker/docker-bench-security
```

* Scan images + host regularly

---

## 🏗️ Supply Chain

* Use **private registries**
* Pin app deps (`requirements.txt`, `package-lock.json`)
* Trust only official orgs for base images

---

## 📊 Health & Logging

```yaml
services:
  api:
    image: myapp
    healthcheck:
      test: ["CMD","curl","-f","http://localhost:8080/health"]
      interval: 10s
      retries: 3
```

* Healthchecks for early failure detection
* Centralized logs, scrub secrets

---

## ✅ Quick Checklist

* [x] Non-root + cap-drop
* [x] Read-only FS
* [x] CPU/RAM/PID limits
* [x] Minimal, pinned base images
* [x] Secrets via env-file/secret manager
* [x] Regular image scans
* [x] Least-open ports
* [x] No Docker socket inside containers

---
